# **PROJETO MVP DA SPRINT DE ENGENHARIA DE DADOS**
- Objetivo (1,0 pt). O objetivo do trabalho deve ser muito bem detalhado; é um planejamento do trabalho, contendo de forma clara e objetiva o problema a ser resolvido e as perguntas de negócio a serem respondidas. Será avaliada a qualidade desta descrição.
- Coleta (0,5 pt). Será avaliada a documentação sobre a coleta dos conjuntos de dados e a persistência dos mesmos na plataforma de nuvem.
- Modelagem (2,0 pt). Será avaliada a qualidade da modelagem dos dados (1,0 pt) e documentação do Catálogo de Dados (1,0 pt).
- Carga (1,0 pt). Será avaliada a qualidade da documentação da carga dos dados, bem como a corretude e persistência dos dados na plataforma de nuvem após a carga.
- Análise (3,0 pt). Serão avaliados a análise de qualidade dos dados (1,0 pt) e da solução do problema de forma correta (0 pt) e bem analisada pela discussão a partir das respostas obtidas (1,0 pt).
- Autoavaliação (0,5 pt). Será avaliada a autoavaliação do aluno com as questões pertinentes sobre o atingimento de seus objetivos traçados no início do trabalho.
- Capricho (2,0 pt). Aqui serão avaliados o capricho e a qualidade geral do trabalho como um todo de forma subjetiva.

# **OBJETIVO**
- Construir uma curva de Phillips para o Brasil com **objetivo** de _1) verificar se existe uma relação de curto prazo inversa entre desemprego e inflação, 2) analisar se é possivel estudar essa relação através de um modelo de regressão linear simples_.

Explicação Teórica: _A Curva de Phillips é um conceito da economia que mostra a relação inversa entre inflação e desemprego no curto prazo. Ela surgiu a partir de estudos do economista A. W. Phillips, que observou que quando o desemprego estava baixo, os salários tendiam a subir mais rapidamente, o que pressionava os preços e aumentava a inflação. Por outro lado, quando o desemprego era alto, os salários cresciam pouco ou até caíam, reduzindo o consumo e levando a uma inflação menor. Assim, a ideia central é que existe um trade-off: políticas que estimulam o emprego podem gerar mais inflação, enquanto medidas para controlar a inflação podem aumentar o desemprego. No entanto, essa relação não é permanente. A partir dos anos 1970, com episódios de estagflação — inflação alta junto com desemprego elevado —, os economistas perceberam que a curva não explicava todos os cenários. Por isso, teorias posteriores, como as de Milton Friedman e Edmund Phelps, argumentaram que no longo prazo a curva se torna vertical, indicando que não é possível reduzir o desemprego apenas aumentando a inflação. Hoje, a Curva de Phillips é vista como uma ferramenta útil para entender os efeitos de políticas monetárias e fiscais, mas com limitações diante da complexidade das economias modernas._



## **Dados**
- ICPA, Série Histórica, Tabela 1737. Fonte: IBGE <https://sidra.ibge.gov.br/Tabela/1737>
- Taxa de desocupação, Tabela 6381. Fonte: IBGE <https://sidra.ibge.gov.br/Tabela/6381>

### **Instalando pacotes e importando bibliotecas**

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf

### **Coleta e Tratamento dos Dados**

- **Coleta**

In [0]:
#Base IPCA
ipca = pd.read_csv(
    'file:/Workspace/Users/m-henrique@live.com/ipca_raw.csv'
)

In [0]:
ipca

In [0]:
#Base Desemprego
desemprego = pd.read_csv(
    'file:/Workspace/Users/m-henrique@live.com/desocupacao_raw.csv'
)

In [0]:
desemprego

- **Tratamento Base IPCA**

In [0]:
#Tratamento Base IPCA
#Realizando a limpeza e manipulação da tabela
ipca = (
        ipca
        .loc[0: ,['V','D2C']]
        .rename(columns = {'V': 'IPCA',
                          'D2C': 'Date'})
)



#tranforma a coluna date em tipo datetime
ipca['date'] = pd.to_datetime(ipca['Date'],
                             format='%Y%m')



#filtra os dados
ipca = ipca.loc[ipca.date >= '2004-01-01']
ipca = ipca.drop(columns=["Date"])



#tranforma a coluna IPCA em tipo float
ipca['IPCA'] = ipca['IPCA'].astype(float)


#Verifica se as alterações funcionaram
ipca.describe()


In [0]:
#Base Pronta
ipca

In [0]:
#grafico do IPCA
sns.lineplot(x='date', y='IPCA', data=ipca)

- **Tratamento Base Desemprego**

In [0]:
#realiza a limpeza e manipulação da tabela
desemprego=(
    desemprego
    .loc[0:,['V', 'D2C']]
    .rename(columns={'V': 'desemprego',
                    'D2C': 'date',}))


#tranforma a coluna date em tipo datetime
desemprego['date']=pd.to_datetime(desemprego['date'], format="%Y%m")


#filtra os dados
desemprego=desemprego.loc[desemprego.date >='2004-01-01']

#tranforma a coluna desocupação em tipo float
desemprego['desemprego']=desemprego['desemprego'].astype(float)

#Verifica se as alterações funcionaram
desemprego.describe()

In [0]:
#Base Pronta
desemprego

In [0]:
#grafico do desemprego
sns.lineplot(x='date', y='desemprego', data=desemprego)

### **Unindo as bases de dados e criando o dataframe que sera utilizado na modelagem**

In [0]:
#juntar dados
df_dados=pd.merge(ipca,desemprego, on='date')
df_dados

## **Modelagem**

In [0]:
#construindo os graficos para analisar se existe uma relação inversa entre os dois dados
plt.plot('date',
        'IPCA',
        data=df_dados,
        label='IPCA')
plt.plot('date',
        'desemprego',
        data=df_dados,
        label='desemprego')
plt.legend()
plt.show()

In [0]:
sns.regplot(x='desemprego',
           y='IPCA',
           data=df_dados)

In [0]:
#Modelo de regressão linear para analisar a relação entre os dados
modelo=smf.ols('IPCA ~ desemprego', data=df_dados).fit()
print(modelo.summary())

## **Análise dos Resultados**

📊 Principais resultados
- Intercepto (constante): 7.016
→ Quando o desemprego é zero (hipoteticamente), a inflação média prevista seria em torno de 7%.
- Coeficiente do desemprego: -0.1201
→ Cada aumento de 1 ponto percentual no desemprego está associado, em média, a uma redução de 0.12 pontos percentuais na inflação.
→ Isso sugere uma relação negativa entre desemprego e inflação (consistente com a ideia da Curva de Phillips).
- Significância estatística (p-valor = 0.084)

📌 Diagnósticos do modelo
- Durbin-Watson = 0.042
→ Extremamente baixo, indicando forte autocorrelação positiva dos resíduos. Isso é um problema: os erros não são independentes, o que compromete a validade dos testes estatísticos.
- Omnibus / Jarque-Bera significativos
→ Os resíduos não seguem distribuição normal (p-valor muito baixo). Isso também afeta a confiabilidade dos intervalos de confiança e testes.
- AIC = 751.2, BIC = 757.3
→ Medidas de qualidade do ajuste. Valores menores indicariam modelos melhores

🧾 Interpretação geral
- Existe indício de uma relação negativa entre desemprego e inflação, mas estatisticamente fraca.
- O modelo sofre com problemas de autocorrelação dos resíduos e não normalidade, o que sugere que uma regressão linear simples não captura bem a dinâmica entre inflação e desemprego.

👉 Em resumo: O modelo sugere, tanto através da analise gráfica quanto do output da regressão que de fato existe uma relação inversa descrita pela curva de Philips entre inflação e desemprego no curto prazo, no entanto os residuos do modelo indicam que a estimação através da técnica de regressão linear não é a melhor metodologia de analise.


### **Catalogo de Dados**

Anexo "_catalogo.txt_"